# Challenge Problem Week 1

Hepatitis B (HEP B) is a liver infection caused by the hepatitis B virus (HBV). The infection causes inflammation of the liver and if not properly treated, the virus can lead to liver disease such as cirrhosis or liver cancer. 
HEP B is the most primary causes of liver cancer, the one of the leading cause of cancer deaths in the world, therfore making it a major global health problem. HEP B is up to 100 times more infectious than the HIV/AIDS virus. Two billion people (1 in 3) have been infected and more than 292 million people are living with a chronic hepatitis B infection. Although HEP B is treatable and preventable about 884,000 people die each year.

The virus is transmitted through the blood and infected bodily fluids. It can be passed to others through direct contact with blood, unprotected sex, use of illegal drugs, unsterilized or contaminated needles, and from an infected woman to her newborn during pregnancy or childbirth. Most people do not show symptoms and the only way to know you are infected is by getting tested.

![hepb](https://images.onhealth.com/images/slideshow/hepatitis-s1-liver-hepatitis-virus.jpg)

**Goal**: Use the NHANES data set to predict whether a patient has HEP B or not. We want to determine which attributes are the most meaningful to the predictive models. We want to create a balanced model that can predict with a high sensitivity and high specificity while using the **least amount of features**. Essentially is there a way to identify the population of those infected without testing them? 

Source: https://www.hepb.org/what-is-hepatitis-b/what-is-hepb/ 

# National Health and Nutrition Examination Survey NHANES 
To investigate our research problem we will be using the NHANES database. NHANES is a program of studies designed to assess the health and nutritional status of adults and children in the United States. The survey is unique in that it combines interviews and physical examinations. The survey examines a nationally representative sample of about 5,000 persons each year. These persons are located in counties across the country, 15 of which are visited each year. The NHANES interview includes demographic, socioeconomic, dietary, and health-related questions. The examination component consists of medical, dental, and physiological measurements, as well as laboratory tests administered by highly trained medical personnel.

Source: Centers for Disease Control and Prevention (CDC). National Center for Health Statistics (NCHS). National Health and Nutrition Examination Survey Data. Hyattsville, MD: U.S. Department of Health and Human Services, Centers for Disease Control and Prevention, [2019][https://www.cdc.gov/nchs/nhanes/about_nhanes.htm#data].

Below are some general steps to begin analyzing this problem. Apply the new material you learned in class and have fun! (:

1. Import the data  
2. Decide what variables are most relevant  
3. Summary statistics of the data  
4. Data Cleaning (Important!) Note this may a tedious process  
a. Missing data  
b. Transform/Normalize data  
4. Data Visualization  
5. Data analysis  
a. Create dummy variables  
b. Create training and test sets  
c. Statistical methodology  
6. Scoring metrics  
confusion matrix, roc curve  


In [2]:
#import needed libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import math
import seaborn as sns


## Import data
Read in the data set and look at the first ten lines

In [3]:
#import data

# Write your code here
dataset = pd.read_csv("NHANES_comp.csv")

/Users/Elaine/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (411,412,414,1973,1974,1975,1976,2595,2597,2599,2601,2603,2605,2607,2609,2611,2613,2615,2617,2619,2621,2623,2625,2627,2629,2631,2633,2635,2637,2639,2641,2643,2645,2647,2649,3143,3194,3331,3332,3345,3346,3349,3350,3351,3352,3353,3526,3545,3863,3865,3867,3869,3871,3873,3875,3877,3879,3881,3883,3885,3887,3889,3891,3893,3895,3897,3899,3901,3903,3905,3907,3909,3911,3913,3915,3917,4161,5018,5047,5077,5081,5423,6581,6582,6584,6598,6599,6601,6855,6857,8094,8572,8573) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#dropping unnecessary variables (don't worry about this)
dataset = dataset[dataset.columns.drop(list(dataset.filter(regex='WT')))]
dataset = dataset[dataset.columns.drop(list(dataset.filter(regex='SDM')))]

Awesome, looks like the data loaded in properly. Let's continue by looking at variables that may be predictive of hepatitis B. For beginners, I would suggest conducting a literature review on previous research of hepatitis B.

## Select Features of Interest
Once you have selected some variables in the NHANES data set only looking at what you are interested in. It is in your best interest to rename the variables.

In [107]:
# Write your code here

# create a subset of the data you want to analyze

df = dataset[['LBXHBC', 'RIDRETH1', 'DUQ370', 'RIDAGEYR']]
df = df.rename(index = str, columns = {'LBXHBC':'Hep B', 'RIDRETH1': 'Ethn', 'DUQ370':'Needle', 'RIDAGEYR': 'Age'})

# rename the variables

Remember the goal is to create a balanced model that can predict with a high sensitivity and high specificity while using the **least amount of features**. Next, we will look at some summary statistics of the variables you chose.

## View summary statistics
Some useful functions in pandas are describe() and info()

In [102]:
# Write your code here
df.head(10)

,Hep B,Ethn,Needle,Age
0,2.0,5,2.0,62
1,2.0,5,NaN,6
2,2.0,3,NaN,71
3,NaN,3,NaN,1
4,2.0,1,2.0,52
5,NaN,1,NaN,6
6,NaN,4,2.0,21
7,2.0,1,2.0,64
8,2.0,4,2.0,66
9,NaN,3,NaN,0


Note the data types are float64, int64 or objects--if there are columns that are obviously numeric like Age but show as objects (or vice versa), we need to change.

## Data Cleaning

Ensure that numeric and categorical variables are coded correctly (turn numeric from strings to numbers).

In [108]:
#drugs: 1 = yes

df = df[np.isfinite(df['Hep B'])]
df = df[np.isfinite(df['Ethn'])]
df = df[df['Needle'] < 7]
df = df[np.isfinite(df['Age'])]

print(df)

       Hep B  Ethn  Needle  Age
0        2.0     5     2.0   62
4        2.0     1     2.0   52
7        2.0     1     2.0   64
8        2.0     4     2.0   66
10       2.0     2     2.0   30
11       2.0     1     2.0   61
12       1.0     5     2.0   27
14       2.0     1     2.0   40
15       2.0     4     2.0   66
19       2.0     1     2.0   40
20       2.0     3     2.0   61
21       2.0     1     2.0   64
23       2.0     4     2.0   68
27       2.0     1     2.0   29
29       2.0     3     2.0   63
34       2.0     2     2.0   48
35       2.0     3     2.0   33
36       2.0     1     2.0   35
43       2.0     3     2.0   50
48       2.0     4     2.0   28
50       2.0     4     1.0   51
51       2.0     3     2.0   64
52       2.0     3     2.0   44
56       2.0     2     2.0   43
61       2.0     3     2.0   44
64       2.0     3     2.0   37
66       2.0     3     2.0   43
67       2.0     4     2.0   60
70       2.0     1     2.0   40
71       2.0     3     2.0   45
...     

Notice the counts for the columns are different because of missing values therefore you will have to figure out how to remediate that issue..some suggestions are found in https://scikit-learn.org/stable/modules/impute.html#impute

In [ ]:
# Write your code here

# remove, impute, some remedial procedure for NAs 


Lastly, we will convert HEP B into indicator variables

In [ ]:
# write your code here


## Data Analysis and Visualization

Take a look at your data. I would suggest doing univariate, bivariate, and multi-variate analysis of most if not all the features you chose.

In [109]:
df['Hep B'] = df['Hep B'].replace(2, 0)
df['Needle'] = df['Needle'].replace(2, 0)
#df['Hep B'] = df['Hep B'].replace(1, 0).replace(2, 1)
#df['Needle'] = df['Needle'].replace(1, 0).replace(2, 1)




print(df)

       Hep B  Ethn  Needle  Age
0        0.0     5     0.0   62
4        0.0     1     0.0   52
7        0.0     1     0.0   64
8        0.0     4     0.0   66
10       0.0     2     0.0   30
11       0.0     1     0.0   61
12       1.0     5     0.0   27
14       0.0     1     0.0   40
15       0.0     4     0.0   66
19       0.0     1     0.0   40
20       0.0     3     0.0   61
21       0.0     1     0.0   64
23       0.0     4     0.0   68
27       0.0     1     0.0   29
29       0.0     3     0.0   63
34       0.0     2     0.0   48
35       0.0     3     0.0   33
36       0.0     1     0.0   35
43       0.0     3     0.0   50
48       0.0     4     0.0   28
50       0.0     4     1.0   51
51       0.0     3     0.0   64
52       0.0     3     0.0   44
56       0.0     2     0.0   43
61       0.0     3     0.0   44
64       0.0     3     0.0   37
66       0.0     3     0.0   43
67       0.0     4     0.0   60
70       0.0     1     0.0   40
71       0.0     3     0.0   45
...     

## Preprocessing data

Before we begin to implement a model, we need to prepare the variables that will be used. At this step we convert categorical variables into dummy\indicator variables (https://chrisalbon.com/python/data_wrangling/pandas_convert_categorical_to_dummies/). Additionally, you'll have to normalize and transform variables if necessary.

In [111]:
# Write your code here

data = pd.get_dummies(df, columns = ['Needle', 'Ethn'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

norm_data = data.Age.values.reshape(-1, 1)
scaler.fit(norm_data)
data['nAge'] = scaler.transform(norm_data)

#data_unitnorm
#sns.distplot(mydat1['nAge'])

data = data.drop(['Age'], axis=1)
data.head()


/Users/Elaine/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Elaine/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,Hep B,Needle_0.0,Needle_1.0,Ethn_1,Ethn_2,Ethn_3,Ethn_4,Ethn_5,nAge
0,0.0,1,0,0,0,0,0,1,1.230398
4,0.0,1,0,1,0,0,0,0,0.520681
7,0.0,1,0,1,0,0,0,0,1.372341
8,0.0,1,0,0,0,0,1,0,1.514284
10,0.0,1,0,0,1,0,0,0,-1.040697


## Model training and selection
Now, let's split our data into training and testing in an 80-20 split, stratified by HEPB distribution (this tries to keep the HEPB distribution approximately equal for the training and test set). For consistency, let's use a random seed 0.

In [113]:
# Write your code here

from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size = 0.20, random_state = 0, stratify = data['Hep B'])


y_train = data_train['Hep B']
y_val = data_val['Hep B']

# only features 
X_train = data_train[["nAge","Ethn_1","Ethn_2", "Ethn_3","Ethn_4", "Ethn_5", "Needle_0.0" , "Needle_1.0"]]
X_val = data_val[["nAge","Ethn_1","Ethn_2", "Ethn_3","Ethn_4", "Ethn_5", "Needle_0.0" , "Needle_1.0"]]


X_train.head()

,nAge,Ethn_1,Ethn_2,Ethn_3,Ethn_4,Ethn_5,Needle_0.0,Needle_1.0
5645,-0.543895,0,0,0,1,0,1,0
2673,-0.898753,1,0,0,0,0,1,0
9510,0.520681,0,1,0,0,0,1,0
5771,-1.040697,0,0,1,0,0,1,0
380,-1.679442,0,0,1,0,0,1,0


Now we can use our training data to create the model and  make predicitons with your test data.

In [118]:
#Write your code here

from sklearn.linear_model import LogisticRegression
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
logreg = LogisticRegression()
results = logreg.fit(X_train, y_train)
from sklearn.metrics import auc

# apply the model to test data
y_val_predict = logreg.predict(X_val)
y_val_proba = logreg.predict_proba(X_val)

print(y_val[:5],y_val_predict[:5])

7931    0.0
1084    0.0
4905    0.0
5263    0.0
9887    0.0
Name: Hep B, dtype: float64 [0. 0. 0. 0. 0.]


/Users/Elaine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Scoring Metrics
## Confusion Matrix Metrics

There are several useful metrics that are derived from the confusion matrix:

![alt text](https://i.imgur.com/uipmEwt.png)

* sensitivity, **recall**, hit rate, or true positive rate (TPR) : $ \mathrm {TPR} ={\frac {\mathrm {TP} }{P}}={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} }}$
 
* **precision** or positive predictive value (PPV) : $ \mathrm {PPV} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FP} }}$

* specificity or true negative rate (TNR) : $\mathrm {TNR} ={\frac {\mathrm {TN} }{N}}={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FP} }}$

* miss rate or false negative rate (FNR) : $ \mathrm {FNR} ={\frac {\mathrm {FN} }{P}}={\frac {\mathrm {FN} }{\mathrm {FN} +\mathrm {TP} }}=1-\mathrm {TPR}$

* fall-out or false positive rate (FPR) : $\mathrm {FPR} ={\frac {\mathrm {FP} }{N}}={\frac {\mathrm {FP} }{\mathrm {FP} +\mathrm {TN} }}=1-\mathrm {TNR} $

* accuracy (ACC) : $\mathrm {ACC} ={\frac {\mathrm {TP} +\mathrm {TN} }{P+N}}={\frac {\mathrm {TP} +\mathrm {TN} }{\mathrm {TP} +\mathrm {TN} +\mathrm {FP} +\mathrm {FN} }}$

Now use code below to calculate the confusion matrix.

In [119]:
# write your code here

from sklearn import metrics
from sklearn.metrics import confusion_matrix

def cm_metric(y_true,y_prob):
    
    # predict the class with the greatest probability
    y_pred = [np.argmax(y) for y in y_prob]

    # calculate the confusion matrix
    cm = confusion_matrix(y_true, y_val_predict)

    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return sum(sum(np.multiply(cm_norm,np.array([[1, -2], [-2, 1]]))))

cm_metric(y_val,y_val_proba)


-0.9373601789709172

The model is classifying everything as class 1... Pretty terrible. :( Well maybe there's a threshold where this doesn't happen. Let's look at the AUC ROC.

## AUC ROC

A receiver operating characteristic (ROC) is a probability curve that plots the true positive rate (y) against the false positive rate (x) at many decision threshold settings. The area under the curve (AUC) represents a measure of separability or how much the model is capable of distinguishing between classes. An AUC closer to 1 is desirable as it shows the model is perfectly distinguishing between patients with disease and no disease. A poor model has an AUC $\leq$ 0.50.

In [ ]:
#extract fpr and tpr to plot ROC curve and calculate AUC (Note: fpr-false positive rate and tpr -true positive rate)
fpr, tpr, threshold = metrics.roc_curve() 

This model only looks at three possible features and leaves lots of room for improvement!  Try using more features, different models, and see if you can do anything about the data we threw out earlier.

## Economic Cost
Similar to the confusion matrix, we want you to keep in mind the other aspects of healthcare analytics--in this case, economic feasibility. In essence, we want you to minimize the amount of time and money spent on data collection by **reducing the number of features** collected. Each record certainly required a lot of time and money from several individuals and businesses to reliably create, and we hope you gain a better understanding of conducting a useful cost-benefit analysis with this scoring method.  This won't be evaluated quantitatively, but please consider discussing it for your presentation.

For your presentation on Friday, don't foget to mention why you selected the features you used, the model implemented, the scoring metrics mentioned above, and the limitations of your analysis.

# Next steps
For those that finish early, try different classification models such as decision trees, KNN, SVM etc. You can try tackling the multiclass classifier (predicting the different cases instead of simply negative or positive)!

Given the rich data set provided feel free to study a research question of your interest. Have fun! (: